## Header

In [2]:
"""
Start file for hw2pr3 of Big Data Summer 2017

The file is seperated into two parts:
	1) the helper functions
	2) the main driver.

The helper functions are all functions necessary to finish the problem.
The main driver will use the helper functions you finished to report and print
out the results you need for the problem.

Before attemping the helper functions, please familiarize with pandas and numpy
libraries. Tutorials can be found online:
http://pandas.pydata.org/pandas-docs/stable/tutorials.html
https://docs.scipy.org/doc/numpy-dev/user/quickstart.html

First, fill in the the code of step 0 in the main driver to load the data, then
please COMMENT OUT any steps in main driver before you finish the corresponding
functions for that step. Otherwise, you won't be able to run the program
because of errors.

After finishing the helper functions for each step, you can uncomment
the code in main driver to check the result.

Note:
1. When filling out the functions below, remember to
	1) Let m be the number of samples
	2) Let n be the number of features

2. Please read the instructions and hints carefully, and use the name of the
variables we provided, otherwise, the function may not work.

3. Remember to comment out the TODO comment after you finish each part.
"""
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time

## Driver

In [70]:

###########################################
#	    	Main Driver Function       	  #
###########################################

# =============STEP 0: LOADING DATA=================
print('==> Loading data...')

# Read data
df = pd.read_csv('https://math189sp19.github.io/data/online_news_popularity.csv', \
    sep=', ', engine='python')

print('==> Successfully Loaded data...')

# split the data frame by type: training, validation, and test
train_pct = 2.0 / 3
val_pct = 5.0 / 6

df['type'] = ''
df.loc[:int(train_pct * len(df)), 'type'] = 'train'
df.loc[int(train_pct * len(df)) : int(val_pct * len(df)), 'type'] = 'val'
df.loc[int(val_pct * len(df)):, 'type'] = 'test'


# extracting columns into training, validation, and test data
X_train = np.array(df[df.type == 'train'][[col for col in df.columns \
    if col not in ['url', 'shares', 'type']]])
y_train = np.array(np.log(df[df.type == 'train'].shares)).reshape((-1, 1))

X_val = np.array(df[df.type == 'val'][[col for col in df.columns \
    if col not in ['url', 'shares', 'type']]])
y_val = np.array(np.log(df[df.type == 'val'].shares)).reshape((-1, 1))

X_test = np.array(df[df.type == 'test'][[col for col in df.columns \
    if col not in ['url', 'shares', 'type']]])
y_test = np.array(np.log(df[df.type == 'test'].shares)).reshape((-1, 1))



# HINT:
# 	1) Use np.ones / np.ones_like to create a column of ones
#	2) Use np.hstack to stack the column to the matrix
"*** YOUR CODE HERE ***"

X_train = np.hstack((np.ones((X_train.shape[0],1)),X_train))
X_val = np.hstack((np.ones((X_val.shape[0],1)),X_val))
X_test = np.hstack((np.ones((X_test.shape[0],1)),X_test))

"*** END YOUR CODE HERE ***"

# Convert data to matrix
X_train = np.matrix(X_train)
y_train = np.matrix(y_train)
X_val = np.matrix(X_val)
y_val = np.matrix(y_val)
X_test = np.matrix(X_test)
y_test = np.matrix(y_test)



# PART C
# =============STEP 1: RMSE vs lambda=================
# NOTE: Fill in code in linreg, findRMSE, and RMSE_vs_lambda for this step

print('==> Step 1: RMSE vs lambda...')

# find the optimal regularization parameter
reg_opt = RMSE_vs_lambda(X_train, y_train, X_val, y_val)
print('==> The optimal regularization parameter is {reg: 4.4f}.'.format(\
    reg=reg_opt))

# Find the optimal weights and bias for future use in step 3
W_with_b_1 = linreg(X_train, y_train, reg=reg_opt)
b_opt_1 = W_with_b_1[0]
W_opt_1 = W_with_b_1[1: ]

# Report the RMSE with the found optimal weights on validation set
val_RMSE = find_RMSE(W_with_b_1, X_val, y_val)
print('==> The RMSE on the validation set with the optimal regularization parameter is {RMSE: 4.4f}.'.format(\
    RMSE=val_RMSE))

# Report the RMSE with the found optimal weights on test set
test_RMSE = find_RMSE(W_with_b_1, X_test, y_test)
print('==> The RMSE on the test set with the optimal regularization parameter is {RMSE: 4.4f}.'.format(\
    RMSE=test_RMSE))



# =============STEP 2: Norm vs lambda=================
# NOTE: Fill in code in norm_vs_lambda for this step

print('\n==> Step 2: Norm vs lambda...')
norm_vs_lambda(X_train, y_train, X_val, y_val)




==> Loading data...
==> Successfully Loaded data...
==> Step 1: RMSE vs lambda...
==> Plotting completed.
==> The optimal regularization parameter is  8.5977.
==> The RMSE on the validation set with the optimal regularization parameter is  0.8340.
==> The RMSE on the test set with the optimal regularization parameter is  0.8628.

==> Step 2: Norm vs lambda...
==> Plotting completed.


## Part C Helpers

In [44]:
np.random.uniform(0.0,100.0)

39.6063682299519

In [69]:

def linreg(X, y, reg=0.0):
    """	This function takes in three arguments:
            1) X, the data matrix with dimension m x (n + 1)
            2) y, the label of the data with dimension m x 1
            3) reg, the parameter for regularization

        This function calculates and returns the optimal weight matrix, W_opt.

        HINT: Find the numerical solution for part C
            1) use np.eye to create identity matrix
            2) use np.linalg.solve to solve for W_opt
    """
    "*** YOUR CODE HERE ***"

    n = X.shape[1] - 1

    #Use the formula from part c
    weird = reg*np.eye(n+1)
    weird[0] = 0
    W_opt = np.linalg.solve(np.dot(np.transpose(X),X) + weird,np.dot(np.transpose(X),y))


    "*** END YOUR CODE HERE ***"
    return W_opt

def predict(W, X):
    """	This function takes in two arguments:
            1) W, a weight matrix with bias
            2) X, the data with dimension m x (n + 1)

        This function calculates and returns the predicted label, y_pred.

        NOTE: You don't need to change this function.
    """
    return X * W


def find_RMSE(W, X, y):
    """	This function takes in three arguments:
            1) W, a weight matrix with bias
            2) X, the data with dimension m x (n + 1)
            3) y, the label of the data with dimension m x 1

        This function calculates and returns the root mean-squared error, RMSE
    """
    "*** YOUR CODE HERE ***"
    
    #Get prediction and subtract from actual
    err = y - predict(W,X)
    
    #Now square everything
    RMSE = np.linalg.norm(err)/np.sqrt(X.shape[0])
    
    

    "*** END YOUR CODE HERE ***"
    return RMSE



def RMSE_vs_lambda(X_train, y_train, X_val, y_val):
    """	This function takes in four arguments:
            1) X_train, the training data with dimension m x (n + 1)
            2) y_train, the label of training data with dimension m x 1
            3) X_val, the validation data with dimension m x (n + 1)
            4) y_val, the label of validation data with dimension m x 1

        This function generates a plot of RMSE vs lambda and returns the
        regularization parameter that minimizes RMSE, reg_opt.

        HINT: get a list of RMSE following the steps below:
            1) Constuct reg_list, a list of regularization parameters with
               random uniform sampling
            2) Generate W_list, a list of W_opt's according to regularization
               parameters generated above
            3) Generate, RMSE_list, a list of RMSE according to reg_list
    """
    RMSE_list = []
    reg_list = [0]*1000
    W_list = []
    "*** YOUR CODE HERE ***"
    reg_list = np.sort([np.random.uniform(0.0,150.0) for x in reg_list])
    for reg in reg_list:
        W = linreg(X_train, y_train,reg)
        RMSE = find_RMSE(W,X_val,y_val)
        W_list = W_list + [W]
        RMSE_list = RMSE_list + [RMSE]
    "*** END YOUR CODE HERE ***"

    # Set up plot style
    plt.style.use('ggplot')

    # Plot RMSE vs lambda
    RMSE_vs_lambda_plot, = plt.plot(reg_list, RMSE_list)
    plt.setp(RMSE_vs_lambda_plot, color='red')
    plt.title('RMSE vs lambda')
    plt.xlabel('lambda')
    plt.ylabel('RMSE')
    plt.savefig('RMSE_vs_lambda.png', format='png')
    plt.close()
    print('==> Plotting completed.')

    "*** YOUR CODE HERE ***"
    best = np.argmin(RMSE_list)
    reg_opt = reg_list[best]

    "*** END YOUR CODE HERE ***"
    return reg_opt

def norm_vs_lambda(X_train, y_train, X_val, y_val):
    """	This function takes in four arguments:
            1) X_train, the training data with dimension m x (n + 1)
            2) y_train, the label of training data with dimension m x 1
            3) X_val, the validation data with dimension m x (n + 1)
            4) y_val, the label of validation data with dimension m x 1

        This function generates a plot of norm of the weights vs lambda.

        HINT:
            1) You may reuse the code from RMSE_vs_lambda to generate
               w_list, the list of weights, and reg_list, the list of
               regularization parameters
            2) Then generate norm_list, a list of norm by calculating the
               norm of each weight
    """
    reg_list = [0]*1000
    W_list = []
    norm_list = []
    "*** YOUR CODE HERE ***"
    reg_list = np.sort([np.random.uniform(0.0,150.0) for x in reg_list])
    for reg in reg_list:
        W = linreg(X_train, y_train,reg)
        norm = np.linalg.norm(W)
        W_list = W_list + [W]
        norm_list = norm_list + [norm]

    "*** END YOUR CODE HERE ***"

    # Set up plot style
    plt.style.use('ggplot')

    # Plot norm vs lambda
    norm_vs_lambda_plot, = plt.plot(reg_list, norm_list)
    plt.setp(norm_vs_lambda_plot, color='blue')
    plt.title('norm vs lambda')
    plt.xlabel('lambda')
    plt.ylabel('norm')
    plt.savefig('norm_vs_lambda.png', format='png')
    plt.close()
    print('==> Plotting completed.')